This file contains the code for network analysis in my MPhil brown adipocyte differentiation project.

## Download the PPI data

This code chunk does not work because I don't know how to visually check if the file was downloaded.

In [ ]:
import requests

# Download the latest STRING database protein nodes file (human, v12.0, medium confidence, remove redundant interactions)
url = "https://stringdb-static.org/download/9606.protein.physical.links.detailed.v12.0.min400.onlyAB.csv.gz"
response = requests.get(url)
with open("9606.protein.physical.links.detailed.v12.0.min400.onlyAB.csv.gz", "wb") as f:
    f.write(response.content)


: 